# Test locking on load

In [ ]:
import pytest
import lamindb_setup as ln_setup
from lamindb_setup.dev.cloud_sqlite_locker import Locker

In [ ]:
assert ln_setup.settings.user.handle == "testuser2"

In [ ]:
ln_setup.init(storage="s3://lndb-setup-ci", name="lndb-load-test-lock")
ln_setup.close()

In [ ]:
# lock with some random id
locker = Locker(
    "randuseridtt", storage_root="s3://lndb-setup-ci", name="lndb-load-test-lock"
)

In [ ]:
locker.lock()

In [ ]:
with pytest.raises(RuntimeError):
    ln_setup.load("lndb-load-test-lock")

In [ ]:
locker.unlock()

In [ ]:
ln_setup.load("lndb-load-test-lock")
ln_setup.close()

In [ ]:
# lock as if by testuser1
locker = Locker(
    "DzTjkKse", storage_root="s3://lndb-setup-ci", name="lndb-load-test-lock"
)

In [ ]:
locker.lock()

In [ ]:
with pytest.raises(RuntimeError):
    ln_setup.load("lndb-load-test-lock")

In [ ]:
locker.unlock()

In [ ]:
ln_setup.load("lndb-load-test-lock")

In [ ]:
ln_setup.delete("lndb-load-test-lock", force=True)